In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd 
import numpy as np 
df1 = pd.read_csv('D://RPI/Daily_files/Datathon/complaints_data/complaints.csv')

In [ ]:
df = df1.copy()

In [ ]:
df.loc[df['Product']=='Credit reporting','Product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product']=='Credit card','Product'] = 'Credit card or prepaid card'
df.loc[df['Product']=='Bank account or service','Product'] = 'Checking or savings account'
df.loc[df['Product']=='Consumer Loan','Product'] = 'Payday loan, title loan, or personal loan'
df.loc[df['Product']=='Payday loan','Product'] = 'Payday loan, title loan, or personal loan'
df.loc[df['Product']=='Money transfers','Product'] = 'Money transfer, virtual currency, or money service'
df.loc[df['Product']=='Prepaid card','Product'] = 'Credit card or prepaid card'
df.loc[df['Product']=='Virtual currency','Product'] = 'Money transfer, virtual currency, or money service'

In [ ]:
df['Product'].value_counts()

In [ ]:
#a = df.loc[df['Product']=='Mortgage',:]
#a.drop(columns = ['Consumer complaint narrative','Company public response'])

In [ ]:
#df.to_csv('D://RPI/Daily_files/Datathon/complaints_data/complaints2_0.csv', sep = ';')
#a.to_csv('D://RPI/Daily_files/Datathon/complaints_data/complaints_mortage.csv')


In [132]:
#date features
import time
from datetime import datetime
df['Date received'] = pd.to_datetime(df['Date received'], format = '%Y-%m-%d')
df['Date sent to company'] = pd.to_datetime(df['Date sent to company'], format = '%Y-%m-%d')

#only use data after 20170501
b = df.loc[df['Date received'] >='2017-05-01',:]

#only use top 5 product data
b = b[b['Product'].isin(['Credit reporting, credit repair services, or other personal consumer reports','Mortgage','Debt collection','Credit card or prepaid card','Checking or savings account','Company public response'])]
c = b.copy()

#drop nlp data and features with much null values
b = b.drop(columns = ['Consumer complaint narrative','Consumer disputed?','Tags'])
#rename the products
b.loc[b['Product']=='Credit reporting, credit repair services, or other personal consumer reports','Product'] = 'CreditRpt'


In [133]:
c = c.loc[c['Product']=='Credit reporting, credit repair services, or other personal consumer reports',:]

In [ ]:
b['Product'].value_counts()

In [143]:
#b['Product'].value_counts()
#print(b.isna().sum())
#print(len(b))
b.info()

#b['Sub-product'].value_counts()
#b['Issue'].value_counts()
#b['Sub-issue'].value_counts()
#b['Company'].value_counts()
#b['Product'].value_counts()
#b['Company response to consumer'].value_counts()
#b['Timely response?'].value_counts()

#b.to_csv('complaints_top5product.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 706853 entries, 0 to 1548291
Data columns (total 15 columns):
Date received                   706853 non-null datetime64[ns]
Product                         706853 non-null object
Sub-product                     706853 non-null object
Issue                           706853 non-null object
Sub-issue                       626253 non-null object
Company public response         371752 non-null object
Company                         706853 non-null object
State                           689807 non-null object
ZIP code                        619969 non-null object
Consumer consent provided?      564791 non-null object
Submitted via                   706853 non-null object
Date sent to company            706853 non-null datetime64[ns]
Company response to consumer    706852 non-null object
Timely response?                706853 non-null object
Complaint ID                    706853 non-null int64
dtypes: datetime64[ns](2), int64(1), object(12)


In [144]:
#about nlp data- c
#c = c['Consumer complaint narrative']
#c.dropna(inplace = True)

c.to_csv('D://RPI/Daily_files/Datathon/complaints_data/narrative.txt', sep='\t', index=False)

D:\programming apps\anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


In [ ]:
#nlp1
#https://blog.csdn.net/HuangZhang_123/article/details/80277793
def getText():
    txt=open("D://RPI/Daily_files/Datathon/complaints_data/narrative.txt","r",encoding='utf-8').read() #打开文件
    txt=txt.lower()                  #将所有单词转换为小写去掉大小写的干扰
    for ch in '`!@#~$%^&*()_+-=*/{}[];,./?<>': #去掉所有的特殊符号
        txt=txt.replace(ch," ")   #将特殊符号替换成空格 即去掉
    return txt
hmltTxt=getText()    #对文件进行读取
words=hmltTxt.split()
#因为现在单词间均为空格分隔开来，所以用split用空格分隔他们并变成列表返回
counts={} #建立一个字典
for word in words:
    counts[word]=counts.get(word,0)+1
    #用当前的某一个单词作为键索引字典 如果在里面则返回次数再加一 若不在里面则直接加1
items=list(counts.items())
#用list将counts变为一个列表类型  counts.items()-->返回可遍历的（键，值）元组数组
items.sort(key=lambda x:x[1],reverse=True)
#使用list.sort()方法来排序，此时list本身将被修改
for i in range(100):
    word,count=items[i]         
    print("{0:<10}{1:>5}".format(word,count))


In [ ]:
#nlp2 
#https://blog.csdn.net/qq_41839921/article/details/83653126
import re
import functools
import pandas as pd
import csv
import os, sys
 
# 把文本内容整理成一个word list
txt=open('D://RPI/Daily_files/Datathon/complaints_data/narrative.txt')
readl=txt.readline()
 
 
r='[’!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]+'
word_list2 = []
while readl:
    readl = txt.readline()
    ff = str(readl)
    ff=re.sub(r,' ',ff) 
    words=ff.split(' ') 
    for word in words:
        word_list2.append(word)    
#print(word_list2)
txt.close()
 
 
#统计频次
tf = {}
for word in word_list2:
    word = word.lower()
        # print(word)
    word = ''.join(word.split())
    if word in tf:
        tf[word] += 1
    else:
        tf[word] = 1
print(tf) 

In [ ]:
#nlp3 https://blog.csdn.net/tanlangqie/article/details/80493888
!pip install jieba

In [ ]:
#nlp3 https://blog.csdn.net/tanlangqie/article/details/80493888
# - * - coding: utf - 8 -*-
#fun: 利用jieba分词和wordcloud创建分词与词用
# time：2018-5-29
# author： tang
# reference： http://blog.csdn.net/fontthrone

from os import path
from scipy.misc import imread
import matplotlib.pyplot as plt
import jieba
# jieba.load_userdict("txt\userdict.txt")
# 添加用户词库为主词典,原词典变为非主词典
from wordcloud import WordCloud, ImageColorGenerator

# 获取当前文件路径
# __file__ 为当前文件, 在ide中运行此行会报错,可改为
# d = path.dirname('.')
d = path.dirname('D://RPI/Daily_files/Datathon/complaints_data/narrative.txt')

stopwords = {}
isCN = 0 #默认启用中文分词
back_coloring_path = r"./Image/narrative_pic.jpg" # 设置背景图片路径
text_path = r'D://RPI/Daily_files/Datathon/complaints_data/narrative.txt' #设置要分析的文本路
#font_path = r'./fonts\simkai.ttf' # 为matplotlib设置中文字体路径没
stopwords_path = r'D://RPI/Daily_files/Datathon/complaints_data/stopword111.txt' # 停用词词表
imgname1 = r"D://RPI/Daily_files/Datathon/complaints_data/WordCloudDefautColors.png" # 保存的图片名字1(只按照背景图片形状)
imgname2 = r"D://RPI/Daily_files/Datathon/complaints_data/WordCloudColorsByImg.png"# 保存的图片名字2(颜色按照背景图片颜色布局生成)

#my_words_list = ['一夜爆红'] # 在结巴的词库中添加新词

back_coloring = imread(r"D://RPI/Daily_files/Datathon/complaints_data/bg.jpg")# 设置背景图片-----back_coloring 为3维数组

# 设置词云属性
#wc = WordCloud(font_path='黑体.ttf',  # 设置字体
#               background_color="white",  # 背景颜色
#               max_words=2000,  # 词云显示的最大词数
#               mask=back_coloring,  # 设置背景图片
#               max_font_size=100,  # 字体最大值
#               min_font_size=20,
 #              random_state=42,
 #              width=1000, height=860, margin=2,# 设置图片默认的大小,但是如果使用背景图片的话,
                                    # 那么保存的图片大小将会按照其大小保存,margin为词语边缘距离
#               )

# 添加自己的词库分词，例如词库中没有一夜爆红这个词，我们可以添加：一夜爆红
#def add_word(list):
#    for items in list:
#        jieba.add_word(items)

#add_word(my_words_list)

text = open( text_path, encoding='utf-8').read()      #打开文本，获取内容

def jiebaclearText(text):
    mywordlist = []                                #存放最终分词结果
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list)                      #未经处理的文本分词结果列表
    f_stop = open(stopwords_path, encoding='utf-8')     #打开停用词词表
    try:
        f_stop_text = f_stop.read( )                      #获取停用词词表中的内容
    finally:
        f_stop.close( )
    f_stop_seg_list = f_stop_text.split('\n')
    for myword in liststr.split('/'):      #获取初次分词结果中的每一个词
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist)

if isCN:   #开启中文分词
    text = jiebaclearText(text)         #获得中文分词结果

# 生成词云, 可以用generate输入全部文本(wordcloud对中文分词支持不好,建议启用中文分词),
# 也可以我们计算好词频后使用generate_from_frequencies函数
wc.generate(text)
wc.generate_from_frequencies(txt_freq)
# txt_freq例子为[('词a', 100),('词b', 90),('词c', 80)]

#image_colors = ImageColorGenerator(back_coloring)       # 从背景图片生成颜色值

plt.figure()
# 以下代码只显示--------形状与背景图片一致，颜色为默认颜色的词云
plt.imshow(wc)
plt.axis("off")
plt.show()     # 绘制词云
wc.to_file(imgname1)    # 保存图片


# 以下代码显示--------形状与背景图片一致，颜色也与背景图颜色一致的词云
image_colors = ImageColorGenerator(back_coloring)        # 从背景图片生成颜色值
plt.imshow(wc.recolor(color_func=image_colors))
plt.axis("off")
plt.show()
wc.to_file( imgname2)


# 绘制以背景图片为颜色的图片----类似于绘制背景图片
# plt.figure()
# plt.imshow(back_coloring, cmap=plt.cm.gray)
# plt.axis("off")
# plt.show()
# 保存图片



In [ ]:
# 词云展示
!pip install nltk
!pip install pillow
!pip install lxml
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from lxml import etree
from nltk.tokenize import word_tokenize



# 去掉停用词
def remove_stop_words(f):

    f_stop = open('D://RPI/Daily_files/Datathon/complaints_data/stopword111.txt', encoding='utf-8')
    try:
        f_stop_text = f_stop.read( )                      #获取停用词词表中的内容
    finally:
        f_stop.close( )
    stop_words = f_stop_text.split('\n')
     
    for stop_word in stop_words:
        f = f.replace(stop_word, '')
    return f

# 生成词云
def create_word_cloud(f):
    print('根据词频，开始生成词云!')
    f = remove_stop_words(f)
    cut_text = word_tokenize(f)
    #print(cut_text)
    cut_text = " ".join(cut_text)
    wc = WordCloud(
        max_words=100,
        width=2000,
        height=1200,
    )
    wordcloud = wc.generate(cut_text)
	# 写词云图片
    wordcloud.to_file("wordcloud.jpg")
    # 显示词云文件
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

# 数据加载

def getText():
    txt=open("D://RPI/Daily_files/Datathon/complaints_data/narrative.txt","r",encoding='utf-8').read() #打开文件
    txt=txt.lower()                  #将所有单词转换为小写去掉大小写的干扰
    for ch in '`!@#~$%^&*()_+-=*/{}[];,./?<>': #去掉所有的特殊符号
        txt=txt.replace(ch," ")   #将特殊符号替换成空格 即去掉
    return txt
hmltTxt=getText()    #对文件进行读取
all_word=hmltTxt.split()

# 生成词云
create_word_cloud(text)

根据词频，开始生成词云!


In [ ]:
#只需要product-creditreporting下面的narrative
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from lxml import etree
from nltk.tokenize import word_tokenize



# 去掉停用词
def remove_stop_words(f):

    f_stop = open('D://RPI/Daily_files/Datathon/complaints_data/stopword111.txt', encoding='utf-8')
    try:
        f_stop_text = f_stop.read( )                      #获取停用词词表中的内容
    finally:
        f_stop.close( )
    stop_words = f_stop_text.split('\n')
     
    for stop_word in stop_words:
        f = f.replace(stop_word, '')
    return f

# 数据加载

def getText():
    txt=open("D://RPI/Daily_files/Datathon/complaints_data/narrative.txt","r",encoding='utf-8').read() #打开文件
    txt=txt.lower()                  #将所有单词转换为小写去掉大小写的干扰
    for ch in '`!@#~$%^&*()_+-=*/{}[];,./?<>': #去掉所有的特殊符号
        txt=txt.replace(ch," ")   #将特殊符号替换成空格 即去掉
    return txt
hmltTxt=getText()    #对文件进行读取
all_word=hmltTxt.split()
